# Pre-processing

In [1]:
from IPython.display import display
import pandas as pd
import re
import numpy as np


DIR_RAW = "../data/raw/"
DIR_CUR = "../data/curated/"


df_postcode = pd.read_csv(f"{DIR_RAW}postcodes.csv")
df_postcode = df_postcode[df_postcode["state"] == "VIC"]
suburbs = set(map(lambda x: x.lower(), df_postcode["locality"].unique()))


PATTERN_PRICE = r"\$?\s*(\d[\d\.,]+)(([\s\/]*((per[\s\/]week)|(weekly)|(p[\/.]*w[k\.]*)|(wk)|(a week)|(w)|(week)|(p\/week)|(per weekly)|(per wk))\b)|$)"
PATTERN_BED = r"^(\d+) beds?"
PATTERN_BATH = r"^(\d+) baths?"
PATTERN_CAR = r"^(\d+) parking"
PATTERN_STATE = r".+ (\w+) \d{4}"
PATTERN_SUBURB = f"({'|'.join(suburbs)}|sanctuary lakes)( vic)?"
PATTERN_BOND = r"bond \$?(\d+)"
PATTERN_INTERNAL_AREA = r"internal area ([\d\.]+)m"
PATTERN_LAND_AREA = r"land area ([\d\.]+)m"
PATTERN_LAST_SOLD = r"last sold in (\d{4})"
PATTERN_OTHER_SOLD = r"(\d+) other"
PATTERN_FIRST_LISTED = r"first listed on (\d+ \w+),"
PATTERN_POSTCODE = r"vic (\d{4})"
PATTERN_PERCENTAGE = r"(\d+\.?\d*)"
PATTERN_PERFOMANCE_PRICE = r"(\d+\.?\d*[mk]?)"
PATTERN_INT = r"([\d,]+)"
PATTERN_RANGE = r"(\d+ to \d+)|(\d+\+)"

FUNC_NONE = lambda x: x
FUNC_STR_TO_NUM = lambda x: float(x.replace(",", ""))
FUNC_PRICE_CONVERT = lambda x: (float(x[0:-1])*1000000 if x[-1] in "mM"
                                else float(x[0:-1])*1000 if x[-1] in "kK"
                                else float(x))
FUNC_PERCENTAGE = lambda x:float(x) / 100


# show all attributes when displayed and don't truncate values
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)


def pattern_match(df, feature, pattern, function=FUNC_NONE):
    instances = df[feature]

    values = []
    for instance in instances:
        instance = str(instance).lower()
        match = re.search(pattern, instance, flags=re.IGNORECASE)
        if match:
            values.append(function(match.group(1)))
        else:
            values.append(None)
    
    return values

In [2]:
df_raw = pd.read_csv(f"{DIR_RAW}scraped_properties.csv")
df_cur = pd.DataFrame()

## Clean Scrapped Data

In [3]:
df_cur["url"] = df_raw["url"]
df_cur["postcode"] = pattern_match(df_raw, "address", PATTERN_POSTCODE)
df_cur["suburb"] = pattern_match(df_raw, "address", PATTERN_SUBURB)  # takes a long time
df_cur["state"] = pattern_match(df_raw, "address", PATTERN_STATE)

df_cur["weekly_rent"] = pattern_match(df_raw, "price", PATTERN_PRICE, FUNC_STR_TO_NUM)
df_cur["bond"] = pattern_match(df_raw, "bond", PATTERN_BOND, FUNC_STR_TO_NUM)

df_cur["num_beds"] = pattern_match(df_raw, "num_beds", PATTERN_BED, FUNC_STR_TO_NUM)
df_cur["num_baths"] = pattern_match(df_raw, "num_bath", PATTERN_BATH, FUNC_STR_TO_NUM)
df_cur["num_parking"] = pattern_match(df_raw, "num_car", PATTERN_CAR, FUNC_STR_TO_NUM)

df_cur["property_type"] = df_raw["property_type"]

df_cur["internal_area"] = pattern_match(df_raw, "internal_area", PATTERN_INTERNAL_AREA, FUNC_STR_TO_NUM)
df_cur["land_area"] = pattern_match(df_raw, "land_area", PATTERN_LAND_AREA, FUNC_STR_TO_NUM)

df_cur["last_sold"] = pattern_match(df_raw, "domain_says", PATTERN_LAST_SOLD)
df_cur["other_sold_n_bed_suburb"] = pattern_match(df_raw, "domain_says", PATTERN_OTHER_SOLD, FUNC_STR_TO_NUM)
#df_cur["first_listed"] = pattern_match(df_raw, "domain_says", PATTERN_FIRST_LISTED)

df_cur["neighbourhood_under_20"] = pattern_match(df_raw, "neighbourhood_under_20", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_20_to_39"] = pattern_match(df_raw, "neighbourhood_20_to_39", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_40_to_59"] = pattern_match(df_raw, "neighbourhood_40_to_59", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_above_60"] = pattern_match(df_raw, "neighbourhood_above_60", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_long_term_residents"] = pattern_match(df_raw, "neighbourhood_long_term_residents", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_owners"] = pattern_match(df_raw, "neighbourhood_owners", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_renter"] = pattern_match(df_raw, "neighbourhood_renter", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_family"] = pattern_match(df_raw, "neighbourhood_family", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["neighbourhood_single"] = pattern_match(df_raw, "neighbourhood_single", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)

df_cur["performance_median_price"] = pattern_match(df_raw, "performance_median_price", PATTERN_PERFOMANCE_PRICE, FUNC_PRICE_CONVERT)
df_cur["performance_auction_clearance"] = pattern_match(df_raw, "performance_auction_clearance", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["performance_sold_this_year"] = pattern_match(df_raw, "performance_sold_this_year", PATTERN_INT, FUNC_STR_TO_NUM)
df_cur["performance_avg_days_on_market"] = pattern_match(df_raw, "performance_avg_days_on_market", PATTERN_INT, FUNC_STR_TO_NUM)

df_cur["demographic_population"] = pattern_match(df_raw, "demographic_population", PATTERN_INT, FUNC_STR_TO_NUM)
df_cur["demographic_owner"] = pattern_match(df_raw, "demographic_owner", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["demographic_renter"] = pattern_match(df_raw, "demographic_renter", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["demographic_family"] = pattern_match(df_raw, "demographic_family", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["demographic_single"] = pattern_match(df_raw, "demographic_single", PATTERN_PERCENTAGE, FUNC_PERCENTAGE)
df_cur["demographic_average_age"] = pattern_match(df_raw, "demographic_average_age", PATTERN_RANGE)

df_cur["latitude"] = df_raw["latitude"].astype(float)
df_cur["longitude"] = df_raw["longitude"].astype(float)

## Merging Data Sets

In [4]:
API_FEATURES = ["school_duration", "school_distance",
                "park_duration", "park_distance",
                "shop_duration", "shop_distance",
                "train_duration", "train_distance",
                "stop_duration", "stop_distance"]

df_school = pd.read_csv(f"{DIR_CUR}api_data.csv")
df_cur = df_cur.merge(df_school, on="url")

## Merge SA2 datasets
## add SA2 zones to each property

In [5]:
import geopandas as gpd
from shapely.geometry import Point


def find_zone(long, lat, shape_df):
    """ Finds the SA2 value based on the coordinates"""
    if np.isnan(lat) or np.isnan(long):
        return 0
    sa2 = shape_df.loc[shape_df["geometry"].contains(Point(long, lat)), "SA2_CODE21"]
    if len(sa2) != 1:
        return 0
    else: 
        return float(sa2)
    
sf_sa2 = gpd.read_file(f"{DIR_RAW}zones/SA2_2021_AUST_GDA2020.shp")
sf_sa2 = sf_sa2[sf_sa2["STE_NAME21"] == "Victoria"]
sf_sa2 = sf_sa2[sf_sa2["geometry"] != None]
sf_sa2 = sf_sa2[["SA2_CODE21", "geometry"]]
sf_sa2 = sf_sa2.astype({"SA2_CODE21": float})

df_cur["SA2"] = df_cur.apply(lambda x: find_zone(x["longitude"], x["latitude"], sf_sa2), axis=1)    

### Add population 

In [6]:
df_pop = pd.read_csv(f"{DIR_RAW}pop.csv")
df_pop = df_pop[df_pop["TIME_PERIOD: Time Period"] == 2021]
df_pop = df_pop[["ASGS_2021: Region", "OBS_VALUE"]]
df_pop["ASGS_2021: Region"] = df_pop["ASGS_2021: Region"].apply(lambda x: x.split(':')[0])
df_pop = df_pop.rename(columns = {"ASGS_2021: Region": "SA2",
                                  "OBS_VALUE": "population"})
df_pop = df_pop.astype({"SA2": float})

df_cur = df_cur.merge(df_pop, on="SA2")

### Add income

In [7]:
df_income = pd.read_csv(f"{DIR_RAW}2021_income.csv")

df_income = df_income[df_income["TIME_PERIOD: Time Period"] == 2021]
df_income = df_income[["REGION: Region", "OBS_VALUE"]]
df_income["REGION: Region"] = df_income["REGION: Region"].apply(lambda x: x.split(':')[0])
df_income = df_income.rename(columns={"REGION: Region": "SA2",
                                      "OBS_VALUE": "median_weekly_income"})
df_income = df_income.astype({"SA2": float})

df_cur = df_cur.merge(df_income, on="SA2")

## Outlier Detection

### Remove NSW town
Most VIC postcodes start with 3 but here is a NSW town (Barooga NSW 3644) who also starts with 3 so it is oftern confused as a VIC town

In [8]:
print(f"Instances before outlier removal: {len(df_cur.index)}")
df_cur = df_cur[df_cur["state"] != "nsw"]
print(f"Instances after  outlier removal: {len(df_cur.index)}")

Instances before outlier removal: 15247
Instances after  outlier removal: 15246


### Remove carspaces
Any property listed as carspace is removed

In [9]:
print(f"Instances before outlier removal: {len(df_cur.index)}")
df_cur = df_cur[df_cur["property_type"] != "Carspace"]
print(f"Instances after  outlier removal: {len(df_cur.index)}")

Instances before outlier removal: 15246
Instances after  outlier removal: 15237


### Remove outliers based on the lower fence and 2 standard deviations

this just removes extreme outliers where data has been input incorrectly

In [10]:
def box_plot_fences(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    
    return Q1 - 1.5*IQR, Q3 + 1.5*IQR


def std_2_bounds(x):
    mean = x.mean()
    std = x.std()
    
    return mean - 2*std, mean + 2*std


def between(df, col, U, L):
    return df[df[col].isnull() | ((df[col] >= L) & (df[col] <= U))]


TEST = ["school_distance", "park_distance",
        "shop_distance", "train_distance",
        "stop_distance"]


df_test = df_cur.copy(deep=True)

print(f"Instances before outlier removal: {len(df_test.index)}")

L, U = box_plot_fences(df_test["weekly_rent"])[0], std_2_bounds(df_test["weekly_rent"])[1]
df_test =  between(df_test, "weekly_rent", U, L)

for col in TEST:
    before = len(df_test.index)
    #L, U = box_plot_fences(df_test[col])
    L, U = std_2_bounds(df_test[col])
    df_test = between(df_test, col, U, L)
    after = len(df_test.index)
    
    #print(f"{col} | [{L:.3f}, {U:.3f}] |  {before - after}")

print(f"Instances after  outlier removal: {len(df_test.index)}")

df_cur = df_test

Instances before outlier removal: 15237
Instances after  outlier removal: 13416


### Remove outliers based on cook's distance

In [11]:
from sklearn.cluster import KMeans
from statsmodels.stats.outliers_influence import OLSInfluence
import statsmodels.regression.linear_model as lm


PRED = ["num_beds", "num_baths", "num_parking", "bond"] + API_FEATURES

TARG = "weekly_rent"

# impute missing values
df_impute = df_cur.copy(deep=True)
df_impute[PRED + [TARG]] = df_cur[PRED + [TARG]].fillna(df_cur[PRED + [TARG]].mean())

# Fit an ordinary linear model
model = lm.OLS(df_impute[[TARG]], df_impute[PRED])
influence = OLSInfluence(model.fit())

print(f"Instances before outleir removal: {len(df_cur.index)}")
df_cur =  df_cur[influence.cooks_distance[0] < 0.002]
print(f"Instances after  outleir removal: {len(df_cur.index)}")

Instances before outleir removal: 13416
Instances after  outleir removal: 13308


## Save 'Current' Pre-Processed Data
### Save data as properties

In [12]:
print(f"There are {df_cur.shape[0]} rows")
display(df_cur.head(5))
df_cur.to_csv(f"{DIR_CUR}/pre_processed_data.csv", index=False)

There are 13308 rows


,url,postcode,suburb,state,weekly_rent,bond,num_beds,num_baths,num_parking,property_type,internal_area,land_area,last_sold,other_sold_n_bed_suburb,neighbourhood_under_20,neighbourhood_20_to_39,neighbourhood_40_to_59,neighbourhood_above_60,neighbourhood_long_term_residents,neighbourhood_owners,neighbourhood_renter,neighbourhood_family,neighbourhood_single,performance_median_price,performance_auction_clearance,performance_sold_this_year,performance_avg_days_on_market,demographic_population,demographic_owner,demographic_renter,demographic_family,demographic_single,demographic_average_age,latitude,longitude,school_duration,school_distance,park_duration,park_distance,shop_duration,shop_distance,train_duration,train_distance,stop_duration,stop_distance,SA2,population,median_weekly_income
1,https://www.domain.com.au/4-12-maylands-street-albion-vic-3020-16005364,3020,albion,vic,400.0,1738.0,2.0,2.0,1.0,Apartment / Unit / Flat,NaN,NaN,None,25.0,0.19,0.40,0.26,0.15,0.62,0.68,0.32,0.42,0.58,280000.0,NaN,25.0,104.0,4731.0,0.52,0.48,0.44,0.56,20 to 39,-37.776549,144.817980,1.238333,527.2,1.625000,717.5,2.571667,1577.0,2.106667,1215.7,0.710000,532.1,213011328.0,7595,704
2,https://www.domain.com.au/5-44-adelaide-street-albion-vic-3020-16050746,3020,albion,vic,280.0,1218.0,2.0,1.0,1.0,Townhouse,NaN,NaN,2008,16.0,0.26,0.37,0.28,0.10,0.52,0.52,0.48,0.50,0.50,578000.0,0.30,16.0,NaN,4731.0,0.52,0.48,0.44,0.56,20 to 39,-37.780469,144.819923,0.528333,300.0,0.713333,301.2,2.756667,1371.8,2.225000,846.1,0.680000,308.6,213011328.0,7595,704
3,https://www.domain.com.au/10-84-rockbank-rd-ardeer-vic-3022-15978682,3022,rockbank,vic,375.0,1630.0,2.0,1.0,1.0,House,NaN,NaN,2016,9.0,0.23,0.40,0.24,0.13,0.55,0.56,0.44,0.45,0.55,NaN,NaN,9.0,NaN,3099.0,0.67,0.33,0.41,0.59,20 to 39,-37.778214,144.798398,0.853333,355.9,0.623333,246.1,8.535000,5810.5,5.161667,2717.6,1.260000,539.5,213011328.0,7595,704
4,https://www.domain.com.au/24-36-ridley-street-albion-vic-3020-16052264,3020,albion,vic,250.0,1083.0,1.0,1.0,1.0,Apartment / Unit / Flat,NaN,NaN,2017,20.0,0.17,0.47,0.25,0.11,0.34,0.33,0.67,0.41,0.59,212000.0,NaN,20.0,77.0,4731.0,0.52,0.48,0.44,0.56,20 to 39,-37.783483,144.823362,1.113333,370.5,0.713333,356.6,3.218333,1586.1,2.508333,1039.7,0.306667,147.9,213011328.0,7595,704
5,https://www.domain.com.au/1-53-maxweld-street-ardeer-vic-3022-15967584,3022,ardeer,vic,390.0,1695.0,3.0,2.0,1.0,Townhouse,NaN,NaN,2015,37.0,0.25,0.28,0.26,0.20,0.68,0.68,0.32,0.45,0.55,680000.0,0.56,37.0,44.0,3099.0,0.67,0.33,0.41,0.59,20 to 39,-37.780276,144.801447,0.733333,279.9,1.058333,410.2,6.403333,3660.5,5.555000,2932.0,1.100000,401.3,213011328.0,7595,704


### Save data as suburb grouping

In [17]:
NUMERICALS = ["weekly_rent", "bond", "num_beds", "num_baths", "num_parking",
              "internal_area", "land_area",
              "other_sold_n_bed_suburb", "neighbourhood_under_20",
              "neighbourhood_20_to_39", "neighbourhood_40_to_59",
              "neighbourhood_above_60", "neighbourhood_long_term_residents",
              "neighbourhood_owners", "neighbourhood_renter",
              "neighbourhood_family", "neighbourhood_single",
              "performance_median_price", "performance_auction_clearance",
              "performance_sold_this_year", "performance_avg_days_on_market",
              "demographic_population", "demographic_owner",
              "demographic_renter", "demographic_family",
              "demographic_single", "school_distance", "park_distance",
              "shop_distance", "train_distance", "stop_distance", "population", "median_weekly_income"]

#CATEGORICALS = ["property_type", "last_sold", "demographic_average_age"]



df_suburb = df_cur.groupby("suburb")
df_suburb = df_suburb.agg({"url": "count"} | {col: "mean" for col in NUMERICALS})
df_suburb = df_suburb.rename(columns={"url": "count"})

df_suburb = df_suburb[df_suburb["count"] > 16]

display(df_suburb.head(5))
df_suburb.to_csv(f"{DIR_CUR}/suburb_data.csv")

,count,weekly_rent,bond,num_beds,num_baths,num_parking,internal_area,land_area,other_sold_n_bed_suburb,neighbourhood_under_20,neighbourhood_20_to_39,neighbourhood_40_to_59,neighbourhood_above_60,neighbourhood_long_term_residents,neighbourhood_owners,neighbourhood_renter,neighbourhood_family,neighbourhood_single,performance_median_price,performance_auction_clearance,performance_sold_this_year,performance_avg_days_on_market,demographic_population,demographic_owner,demographic_renter,demographic_family,demographic_single,school_distance,park_distance,shop_distance,train_distance,stop_distance,population,median_weekly_income
suburb,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
abbotsford,52,548.600000,2387.020000,2.000000,1.365385,1.195122,NaN,NaN,79.120000,0.107692,0.555577,0.230577,0.104231,0.399231,0.347600,0.652400,0.505000,0.495000,7.622340e+05,0.514255,79.916667,72.510638,8443.680000,0.396000,0.604000,0.259200,0.740800,772.030769,434.905882,1081.582609,1406.405769,398.075000,9780.153846,1323.153846
abeckett street,19,501.842105,2470.909091,1.421053,1.105263,1.000000,NaN,NaN,455.736842,0.063158,0.776842,0.131579,0.033684,0.180714,0.260714,0.739286,0.285714,0.714286,4.430714e+05,0.436429,463.357143,127.357143,44828.428571,0.307143,0.692857,0.242143,0.757857,707.547368,493.621053,773.422222,388.752632,321.731579,17545.368421,675.105263
ada,25,455.416667,2034.720000,2.680000,1.800000,1.500000,NaN,NaN,135.750000,0.209565,0.466522,0.210870,0.107826,0.573000,0.550435,0.449565,0.528261,0.471739,7.270476e+05,0.534706,135.750000,71.761905,13646.280000,0.550000,0.450000,0.391200,0.608800,1815.372000,4185.372000,6630.104762,3456.820000,537.276000,12763.040000,781.040000
airport west,17,428.529412,1909.142857,2.529412,1.294118,1.294118,NaN,299.0,48.588235,0.222353,0.261765,0.262941,0.251765,0.621765,0.721765,0.278235,0.545882,0.454118,7.813750e+05,0.704375,48.588235,70.928571,7567.000000,0.730000,0.270000,0.470000,0.530000,1007.452941,504.223529,1178.320000,7175.617647,260.870588,8237.000000,883.000000
albert park,17,900.882353,4799.375000,2.352941,1.529412,1.100000,NaN,179.0,47.470588,0.260000,0.205294,0.304706,0.227059,0.507647,0.618235,0.381765,0.595882,0.404118,1.839176e+06,0.751818,47.470588,36.454545,6210.000000,0.610000,0.390000,0.470000,0.530000,509.900000,364.858824,2331.300000,4022.794118,247.176471,16023.000000,1352.000000
